# Prototype notebook

## Fetching and indexing

In [30]:
import choral_explanation
from importlib import reload
reload(choral_explanation)

<module 'choral_explanation' from '/Users/sander/Code/choral-explanation/choral_explanation/__init__.py'>

In [18]:
question_list_data = choral_explanation.fetch_question_list_data('tip')
display(str(question_list_data[0])[:80])

"{'nameID': 'toelichting-2833', 'type': 'POST', 'createdBy': {'profile': {'url': "

In [19]:
questions = [choral_explanation.question(data) for data in question_list_data]
display(str(questions[0])[:80])

"{'id': 'toelichting-2833', 'title': 'Toelichting', 'url': 'https://alkem.io/tip/"

In [ ]:
authors = {author
           for question in questions
           for author in choral_explanation.authors(question)}

In [7]:
index = dict(choral_explanation.index_all(questions))
len(index)

211

In [8]:
questions[0]

{'id': 'toelichting-2833',
 'title': 'Toelichting',
 'url': 'https://alkem.io/tip/collaboration/toelichting-2833',
 'description_md': 'Momenteel gebruiken wij Alkemio alleen voor de [Kennisbank](https://alkem.io/tip/knowledge-base). Neem contact op met [Sander Dijkhuis](https://alkem.io/user/sander-dijkhuis-3912) voor vragen.',
 'answers': [],
 'author_name': 'Sander Dijkhuis',
 'author_url': 'https://alkem.io/user/sander-dijkhuis-3912'}

In [22]:
space_data = choral_explanation.fetch_space_data('tip')
display(space_data['community'])

{'memberUsers': None}

## Writing to Git working directory

In [47]:
import os
import shutil

DIR_PATH = "example/tip"
shutil.rmtree(DIR_PATH)
os.makedirs(DIR_PATH, exist_ok=True)


In [73]:
for question in questions:
    choral_explanation.write_github_flavored_md(question, DIR_PATH, index)

## Writing to Obsidian directory

In [67]:
import os
import shutil

DIR_PATH = "/Users/sander/Obsidian/Notes/Replica/TIP Kennisbank"
shutil.rmtree(DIR_PATH, ignore_errors=True)
os.makedirs(f"{DIR_PATH}/Participants", exist_ok=True)

import choral_explanation
from importlib import reload
reload(choral_explanation)

SKIP = {
    'overzichtvanreleva-7668',
    'implementatiekennis-617',
    'overzichtvanreleva-3471',
    'toelichting-2833',
}
for question in questions:
    if question['id'] not in SKIP:
        choral_explanation.write_obsidian_md(question, DIR_PATH, index, authors)
for author_name, author_url in authors:
    choral_explanation.write_author_obsidian_md(author_name, author_url, DIR_PATH)
choral_explanation.write_author_index_obsidian_md(
    choral_explanation.stats(questions),
    sorted([name for name, url in authors], key=str.lower),
    DIR_PATH
)

In [64]:
display(choral_explanation.stats(questions))

{'questions': 111, 'answers': 100, 'authors': 13}